# [Mantichora](https://github.com/alphatwirl/mantichora)

A simple interface to _multiprocessing_ and _threading_

[Tai Sakuma](https://github.com/TaiSakuma)

_updated on March 29, 2020_

_Mantichora_ provides a simple interface to
[_multiprocessing_](https://docs.python.org/3/library/multiprocessing.html) and
[_threading_](https://docs.python.org/3/library/threading.html).

```python
from mantichora import mantichora

with mantichora() as mcore:
    mcore.run(func1)
    mcore.run(func2)
    mcore.run(func3)
    mcore.run(func4)
    results = mcore.returns()
```

You can simply give Mantichora as many functions as you need to run.
Mantichora will run them concurrently in background processes by using
_multiprocessing_ or in different threads by using _threading_ and give
you the return values of the functions. The return values are sorted
in the order of the functions you have originally given to Mantichora.
Progress bars from [atpbar](https://github.com/alphatwirl/atpbar) can
be used in the functions.

The code in this package was originally developed in the sub-package
[_concurrently_](https://github.com/alphatwirl/alphatwirl/tree/v0.23.2/alphatwirl/concurrently)
of [_alphatwirl_](https://github.com/alphatwirl/alphatwirl).

*****

- **GitHub:** https://github.com/alphatwirl/mantichora
- **PyPI:** https://pypi.org/project/mantichora/

*****

- [**Quick start**](#Quick-start)
    - [Import libraries](#Import-libraries)
    - [Define a task function](#Define-a-task-function)
    - [Run tasks concurrently with Mantichora](#Run-tasks-concurrently-with-Mantichora)
- [**Features**](#Features)
    - [Multiprocessing or Threading](#Multiprocessing-or-Threading)
    - [Without the `with` statement](#Without-the-with-statement)
        - [`end()`](#end())
        - [`terminate()`](#terminate())
    - [Receive results as tasks finish](#Receive-results-as-tasks-finish)
        - [`receive_one()`](#receive_one())
        - [`receive_finished()`](#receive_finished())
- [**License**](#License)
- [**Contact**](#Contact)


*****
## Quick start

I will show here how to use Mantichora by simple examples.

### Import libraries

We are going use two python standard libraries
[time](https://docs.python.org/3/library/time.html) and
[random](https://docs.python.org/3/library/random.html) in an example
task function. In the example task function, we are also going to use
[atpbar](https://github.com/alphatwirl/atpbar) for progress bars.
Import these packages and `mantichora`.




In [1]:
import time, random
from atpbar import atpbar
from mantichora import mantichora

### Define a task function

Let us define a simple task function.

In [2]:
def task_loop(name, ret=None):
    n = random.randint(1000, 10000)
    for i in atpbar(range(n), name=name):
        time.sleep(0.0001)
    return ret

The task in this function is to sleep for `0.0001` seconds as many
times as the number randomly selected from between `1000` and
`10000`. `atpbar` is used to show a progress bar. The function takes
two arguments: `name`, the label on the progress bar, and `ret`, the
return value of the function.

**Note:** In the multiprocessing mode, the default mode of mantichora,
task functions, their arguments, and their return values need to be
[picklable](https://docs.python.org/3.8/library/pickle.html#what-can-be-pickled-and-unpickled).

You can just try running this function without using Mantichora. The next cell won't return immediately. It will wait for the function to finish. You will see a progress bar if running interactively. (If you are reading a saved notebook, for exampe, on GitHub, you might just see the string `VBox()` or other text messages instead of the progress bar.)

In [3]:
task_loop('task1', 'result1')

VBox()

'result1'

### Run tasks concurrently with Mantichora

Now, we run multiple tasks concurrently with Mantichora. Progress bars will be shown by `atpbar`.

In [4]:
with mantichora(nworkers=3) as mcore:
    mcore.run(task_loop, 'task', ret='result1')
    mcore.run(task_loop, 'another task', ret='result2')
    mcore.run(task_loop, 'still another task', ret='result3')
    mcore.run(task_loop, 'yet another task', ret='result4')
    mcore.run(task_loop, 'task again', ret='result5')
    mcore.run(task_loop, 'more task', ret='result6')
    results = mcore.returns()

VBox()

In the above cell, `mantichora` is initialized with an
optional argument `nworkers`. The `nworkers` specifies the number of
the workers. It is `3` in the above example. The default is `4`. At
most as many tasks as `nworkers` can run concurrently.

The [`with`
statement](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement)
is used in the example. This ensures that `mantichora` properly
ends the workers.

You can give task functions and their arguments to `mcore.run()`. You
can call `mcore.run()` as many times as you need. In the above
example, `mcore.run()` is called with the same task function with
different arguments. You can also use a different function each time.
`mcore.run()` returns immediately; it doesn't wait for the task to
finish or even to start. In each call, `mcore.run()` only puts a task in
a queue. The workers in background processes pick up tasks from the
queue and run them.

The `mcore.returns()` waits until all tasks finish and returns their
return values, which are sorted in the order of the tasks you have
originally given to `mcore.run()` regardless of the order
in which the tasks have finished.

In [5]:
results

['result1', 'result2', 'result3', 'result4', 'result5', 'result6']

*****
## Features

#### Multiprocessing or Threading
        
*New in version 0.10.0*

From version 0.10.0, you can choose to use
[threading](https://docs.python.org/3/library/threading.html) instead
of
[multiprocessing](https://docs.python.org/3/library/multiprocessing.html)
by setting the option `mode` to `threading` (the default is
`multiprocessing`).


In [6]:
with mantichora(mode='threading') as mcore:
    mcore.run(task_loop, 'task', ret='result1')
    mcore.run(task_loop, 'another task', ret='result2')
    mcore.run(task_loop, 'still another task', ret='result3')
    mcore.run(task_loop, 'yet another task', ret='result4')
    mcore.run(task_loop, 'task again', ret='result5')
    mcore.run(task_loop, 'more task', ret='result6')
    results = mcore.returns()

VBox()

*****

### Without the `with` statement

#### `end()`

If you don't use the `with` statement, you need to call `end()`.

In [7]:
mcore = mantichora()

mcore.run(task_loop, 'task', ret='result1')
mcore.run(task_loop, 'another task', ret='result2')
mcore.run(task_loop, 'still another task', ret='result3')
mcore.run(task_loop, 'yet another task', ret='result4')
mcore.run(task_loop, 'task again', ret='result5')
mcore.run(task_loop, 'more task', ret='result6')

results = mcore.returns()

mcore.end()
results

VBox()

['result1', 'result2', 'result3', 'result4', 'result5', 'result6']

#### `terminate()`

`mantichora` can be terminated with `terminate()`. After `terminate()` is called, `end()` still needs to be called. In the example below, `terminate()` is called after 0.5 seconds of sleep while some tasks are still running.

In [8]:
mcore = mantichora()

mcore.run(task_loop, 'task', ret='result1')
mcore.run(task_loop, 'another task', ret='result2')
mcore.run(task_loop, 'still another task', ret='result3')
mcore.run(task_loop, 'yet another task', ret='result4')
mcore.run(task_loop, 'task again', ret='result5')
mcore.run(task_loop, 'more task', ret='result6')

time.sleep(0.5)

mcore.terminate()
mcore.end()

VBox()

**Note:**: In the threading mode, `terminate()` does not do anything. If you initialize mantichora in the threading mode, i.e., `mantichora(mode='threading')`, in the above cell, all tasks run until completion.

*****

### Receive results as tasks finish

Instead of waiting for all tasks to finish beofre receiving the reulsts, you can get results as tasks finish with the method `receive_one()` or `receive_receive()`.

#### `receive_one()`

The method `receive_one()` returns a pair of the run ID and return value of a task function. If no task has finished, `receive_one()` waits until one task finishes. `receive_one()` returns `None` if no tasks are outstanding. The method `run()` returns the run ID for the task.

In [9]:
with mantichora() as mcore:
    runids = [ ]
    runids.append(mcore.run(task_loop, 'task1', ret='result1'))
    runids.append(mcore.run(task_loop, 'task2', ret='result2'))
    runids.append(mcore.run(task_loop, 'task3', ret='result3'))
    runids.append(mcore.run(task_loop, 'task4', ret='result4'))
    runids.append(mcore.run(task_loop, 'task5', ret='result5'))
    runids.append(mcore.run(task_loop, 'task6', ret='result6'))

    pairs = [ ]
    for i in range(len(runids)):
        pairs.append(mcore.receive_one())

VBox()

The `runid` is the list of the run IDs in the order of the tasks that have been given to `run()`.

In [10]:
runids

[0, 1, 2, 3, 4, 5]

The `pairs` are in the order in which the tasks have finished.

In [11]:
pairs

[(3, 'result4'),
 (1, 'result2'),
 (5, 'result6'),
 (0, 'result1'),
 (2, 'result3'),
 (4, 'result5')]

####  `receive_finished()`

The method `receive_finished()` returns a list of pairs of the run ID and the return value of finished task functions. The method `receive_finished()` doesn't wait for a task to finish. It returns an empty list if no task has finished.

In [12]:
with mantichora() as mcore:
    runids = [ ]
    runids.append(mcore.run(task_loop, 'task1', ret='result1'))
    runids.append(mcore.run(task_loop, 'task2', ret='result2'))
    runids.append(mcore.run(task_loop, 'task3', ret='result3'))
    runids.append(mcore.run(task_loop, 'task4', ret='result4'))
    runids.append(mcore.run(task_loop, 'task5', ret='result5'))
    runids.append(mcore.run(task_loop, 'task6', ret='result6'))

    pairs = [ ]
    while len(pairs) < len(runids):
        pairs.extend(mcore.receive_finished())

VBox()

The `runid` is again the list of the run IDs in the order of the tasks that have been given to `run()`.

In [13]:
runids

[0, 1, 2, 3, 4, 5]

The `pairs` are also again in the order in which the tasks have finished.

In [14]:
pairs

[(1, 'result2'),
 (0, 'result1'),
 (3, 'result4'),
 (4, 'result5'),
 (5, 'result6'),
 (2, 'result3')]

*****

## License

- Mantichora is licensed under the BSD license.

*****

## Contact

- Tai Sakuma - tai.sakuma@gmail.com